In [1]:
%%capture
import os
os.chdir('/tmp')
!wget https://github.com/MontrealCorpusTools/Montreal-Forced-Aligner/releases/download/v1.0.1/montreal-forced-aligner_linux.tar.gz
!tar zxvf montreal-forced-aligner_linux.tar.gz
!ln -s /tmp/montreal-forced-aligner/lib/libpython3.6m.so.1.0 /tmp/montreal-forced-aligner/lib/libpython3.6m.so

In [2]:
os.chdir('/kaggle/working')
os.environ['LD_LIBRARY_PATH'] = f'{os.environ["LD_LIBRARY_PATH"]}:/tmp/montreal-forced-aligner/lib/'
os.environ['PATH'] = f'{os.environ["PATH"]}:/tmp/montreal-forced-aligner/bin/'

In [3]:
%%capture
!yes|apt install libgfortran3

In [4]:
!mkdir /tmp/mfa-temp

In [5]:
import json
datapath = '../input/living-audio-irish-speech-corpus/living-audio.json'
with open(datapath) as jsonf:
    data = json.load(jsonf)

In [6]:
!mkdir /tmp/living-audio

In [7]:
lexicon_words = set()
with open('../input/living-audio-irish-speech-corpus/lexicon.txt') as lexicon_file:
    for line in lexicon_file.readlines():
        words = line.split(' ')
        lexicon_words.add(words[0])

In [8]:
import shutil
missing_words = set()
for utt in data:
    shutil.copyfile(utt['path'], f"/tmp/living-audio/{utt['id']}.wav")
    with open(f"/tmp/living-audio/{utt['id']}.txt", 'w') as text:
        sentence = utt['sentence']
        sentence = sentence.replace('(', '').replace(')', '') 
        words = []
        for word in sentence.split(' '):
            if not word in lexicon_words:
                missing_words.add(word)
            if '-' in word:
                if word.startswith('n-') or word.startswith('t-'):
                    workword = word[2:]
                    workword.replace('-', ' ')
                    word = word[0:2] + workword
                else:
                    word = word.replace('-', ' ')
            words.append(word)
        text.write(' '.join(words))

In [9]:
!mfa_train_and_align -t /tmp/mfa-temp -o ./irish-model /tmp/living-audio ../input/living-audio-irish-speech-corpus/lexicon.txt ./textgrid

Setting up corpus information...
Creating dictionary information...
Setting up training data...
Calculating MFCCs...
Calculating CMVN...
Number of speakers in corpus: 1, average number of utterances per speaker: 1121.0
b'number of phones 215\nnumber of pdfs 165\nnumber of transition-ids 1470\nnumber of transition-states 675\nfeature dimension 39\nnumber of gaussians 165\n'
None
b'number of phones 215\nnumber of pdfs 165\nnumber of transition-ids 1470\nnumber of transition-states 675\nfeature dimension 39\nnumber of gaussians 165\n'
None
Beginning monophone training...
100%|███████████████████████████████████████████| 39/39 [25:35<00:00, 21.13s/it]
Initializing triphone training...
Beginning triphone training...
100%|███████████████████████████████████████████| 34/34 [19:09<00:00, 16.35s/it]
Initializing speaker-adapted triphone training...
Beginning speaker-adapted triphone training...
100%|███████████████████████████████████████████| 34/34 [08:55<00:00,  8.33s/it]
Saved model to ./iri